In [24]:
import petromodder as pm
import xtgeo

In [3]:
pm_mod = pm.PetroMod("//mnt//d//code//petromodder//PM_model//pm3d//LayerCake")

Valid PetroMod model


PetroMod has 2 folder for storing lines. They are the same

In [8]:
#Cross-section line
pm_mod.crossline["lines"]

{'Line_01': <shapely.geometry.linestring.LineString at 0x7f3774d8feb0>}

In [10]:
#Simple Line
pm_mod.polygons["lines"]

{'Line_01': <shapely.geometry.linestring.LineString at 0x7f3774d54340>}

In [41]:
#Get coordinates
coord = list(pm_mod.polygons["lines"]["Line_01"].coords)

In [48]:
pm_mod.lookup_table

,Key,HorizonAge,HorizonID,HorizonName,HorizonColor,LayerID,LayerName,LayerColor,LayerType,NoSubLayer,MaxTimeStep,Unnamed: 11,HorizonGeomID,FaciesMapID
0,Data,0.0,1,Horizon_6,-16777088,1,Layer_5,-16777088,0,1,10.0,NaN,2,1
1,Data,10.0,2,Horizon_5,-16777021,2,Layer_4,-16777021,0,1,10.0,NaN,3,2
2,Data,20.0,3,Horizon_4,-16776961,3,Layer_3,-16776961,0,1,10.0,NaN,4,3
3,Data,30.0,4,Horizon_3,-16749057,4,Layer_2,-16749057,0,1,10.0,NaN,5,4
4,Data,40.0,5,Horizon_2,-16739841,5,Layer_1,-16739841,0,1,10.0,NaN,6,5
5,Data,50.0,6,Horizon_1,-16730369,-1,,-1,-1,-1,99999.0,NaN,7,-1


In [27]:
#Geometry of the PetroMod model
#HorizonAge = Age of the surface grid
#HorizonGeomID = file name of the surface grid
#FaciesMapID = facies map of the surface
#If HorizonGeomID is -1, that means the event is erosional/hiatus. Ignore for now
table = pm_mod.lookup_table[["HorizonAge","HorizonGeomID","FaciesMapID"]]
table

,HorizonAge,HorizonGeomID,FaciesMapID
0,0.0,2,1
1,10.0,3,2
2,20.0,4,3
3,30.0,5,4
4,40.0,6,5
5,50.0,7,-1


In [25]:
#Path to depth map
map_path=pm_mod.path / "in//ts1//bl1//dpth_m//"

In [44]:
#Read the first surface grid
filename = str(int(table.iloc[0]["HorizonGeomID"]))+".pmd"
fullpath = map_path / filename
surface = xtgeo.surface_from_file(fullpath, fformat="petromod")

['Content=Map', 'DataUnitDistance=15', 'DataUnitZ=10', 'GridNoX=101', 'GridNoY=101', 'GridStepX=100', 'GridStepY=100', 'MapType=GridMap', 'OriginX=0', 'OriginY=0', 'RotationAngle=0', 'RotationOriginX=0', 'RotationOriginY=0', 'Undefined=99999', 'Version=1.0']


In [45]:
#Extract depth value from coordinates
l = coord[0]
surface.get_value_from_xy(point=l)

0.0

In [50]:
#Path to facies map
facies_path=pm_mod.path / "in//ts1//bl1//faci_m//"

In [51]:
#Read the first facies map
filename = str(int(table.iloc[0]["FaciesMapID"]))+".pmd"
fullpath = facies_path / filename
facies = xtgeo.surface_from_file(fullpath, fformat="petromod")

['Content=Map', 'DataUnitDistance=15', 'DataUnitZ=300', 'GridNoX=100', 'GridNoY=100', 'GridStepX=100', 'GridStepY=100', 'MapType=CellMap', 'OriginX=0', 'OriginY=0', 'RotationAngle=0', 'RotationOriginX=0', 'RotationOriginY=0', 'Undefined=99999', 'Version=1.0']


In [53]:
#Extract facies value from coordinates
l = coord[0]
facies_code = facies.get_value_from_xy(point=l)

In [55]:
#Convert map facies code into facies code in the PetroMod mode
facies_conv = pm_mod.facies_table
if pm_mod.version == "2017.1":
    facies_conv = facies_conv[["ID", "LithValue"]]
    facies_conv = facies_conv.set_index("ID").T.to_dict("int")
    facies_conv = facies_conv["LithValue"]
elif pm_mod.version == "2019.1":
    facies_conv = facies_conv[["ID", "LithoUUID"]]
    facies_conv = facies_conv.set_index("ID").T.to_dict("int")
    facies_conv = facies_conv["LithoUUID"]

In [58]:
#Get the facies code and thermal properties dictionary
#The xml file will only exist if user has created custom lithology
lith = pm_mod.lithology
facies = {}
for k, v in facies_conv.items():
    map_k = str(k)
    lithval = str(v)
    d = {map_k: lith[lithval]}
    facies.update(d)

In [66]:
facies[str(int(facies_code))]

{'Thermal Conduct. Model Key': '1',
 'Thermal Conduct. at 20°C': '1.64',
 'Thermal Conduct. at 100°C': '1.69',
 'Thermal Conduct. Max. Temperature': '593.15',
 'Thermal Conduct. Multi-Point Model': '38aa3330-1ac6-46e9-aa4d-3772d73a5cb5',
 'Anisotropy Factor Thermal Conduct.': '1.6',
 'Depositional Anisotropy (On/Off)': 'True',
 'Depositional Anisotropy': '1.02',
 'Thermal Expansion Coefficient': '3.3e-5',
 'Uranium': '3.7e-6',
 'Thorium': '1.2e-5',
 'Potassium': '0.027',
 'Heat Capacity Model': '4',
 'Heat Capacity at 20°C': '862.4808',
 'Heat Capacity at 100°C': '996.4584',
 'Heat Capacity Max. Temperature': '593.15',
 'Heat Capacity Curve': 'e8b0c67f-8dc6-4aee-b989-04bbad3575d4',
 'Density': '2700',
 'Initial Porosity': '0.7',
 'Minimum Porosity': '0.01',
 'Compaction Model Key': '5',
 'Compressibility Max': '4.0327e-7',
 'Compressibility Min': '4.03e-9',
 "Athy's Factor k (stress)": '9.613e-8',
 'Schneider Factor ka': '1.9157e-7',
 'Schneider Factor kb': '5.27e-8',
 'Schneider Facto